In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow import keras
import sys
import PIL
import os
import tensorflow.keras.backend as K
import glob
sys.path.append('../scripts/helper_functions_cv/tensorflow_helpers/')
from dotenv import load_dotenv
from sklearn.model_selection import train_test_split
import segmentation_models as sm
sm.set_framework('tf.keras')
import multiprocessing as mp
from save_weights_every_epoch import CallbackForSavingModelWeights
import math

Segmentation Models: using `keras` framework.


In [2]:
csv_file = pd.read_csv('../../ml-data-training/ship_segmentation_data/train_ship_segmentations_v2.csv')
csv_file = csv_file.groupby('ImageId')['EncodedPixels'].apply(list).reset_index()
image_ids, pixels = csv_file['ImageId'].values.tolist(), csv_file['EncodedPixels'].values.tolist()

In [3]:
csv_file['fixed_inputs'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/train_v2/' + x)
csv_file['mask_paths'] = csv_file['ImageId'].apply(lambda x: '../../ml-data-training/ship_segmentation_data/masks_v1/train/' + x.split('.')[0] + '.' + 'png')

In [4]:
# for x in csv_file['fixed_inputs'].values.tolist():
#     if os.path.exists(x) == False:
#         print(x)

In [5]:
# for x in csv_file['mask_paths'].values.tolist():
#     if os.path.exists(x) == False:
#         print(x)

In [6]:
csv_file = csv_file[csv_file['fixed_inputs'] != '../../ml-data-training/ship_segmentation_data/train_v2/6384c3e78.jpg']

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [8]:
allowed_gpus = [0]
gpus = tf.config.list_physical_devices("GPU")
final_gpu_list = [gpus[x] for x in allowed_gpus]
tf.config.set_visible_devices(final_gpu_list, "GPU")

strategy = tf.distribute.MirroredStrategy()
AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


In [9]:
print(REPLICAS)

1


In [10]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
load_dotenv('../config_files/dev.env')

True

In [12]:
def split_datasets(csv_file, test_size = 0.01):
    train, test = train_test_split(csv_file, test_size = test_size)
    train, val = train_test_split(train, test_size = test_size)
    return train, val, test

In [13]:
train, val, test = split_datasets(csv_file)

In [14]:
train.head()

,ImageId,EncodedPixels,fixed_inputs,mask_paths
168238,dfc4b5e71.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
75370,643f953d3.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
55541,4a06f9233.jpg,[357869 1 358636 3 359404 5 360171 7 360939 8 ...,../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
3357,04845f032.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...
183430,f3f4f6c7f.jpg,[nan],../../ml-data-training/ship_segmentation_data/...,../../ml-data-training/ship_segmentation_data/...


In [15]:
def read_train_imgs(img, mask, shape):
    img = tf.io.read_file(img)
    mask = tf.io.read_file(mask)
    img = tf.image.decode_jpeg(img, channels=3)
    mask = tf.image.decode_jpeg(mask, channels=1)
    img = tf.image.resize(img, size=shape)
    mask = tf.image.resize(mask, size=shape)
    img = img / 255
    mask = mask / 255
    return img, mask

def get_data(data, shape = (256, 256), shuffle = True, repeat = True, batch = True, batch_size = 32):
    imgs, masks = data['fixed_inputs'].values.tolist(), data['mask_paths'].values.tolist()
    shapes = [shape for x in range(len(imgs))]
    tensor = tf.data.Dataset.from_tensor_slices((imgs, masks, shapes))
    tensor = tensor.cache()
    if repeat:
        tensor = tensor.repeat()
    if shuffle:
        tensor = tensor.shuffle(256 * REPLICAS)
        opt = tf.data.Options()
        opt.experimental_deterministic = False
        tensor = tensor.with_options(opt)
    tensor = tensor.map(read_train_imgs)
    if batch:
        tensor = tensor.batch(batch_size * REPLICAS)
    tensor = tensor.prefetch(AUTO)
    return tensor

In [19]:
train_dataset = get_data(train, shape = (256, 256), batch_size=64, shuffle=False, repeat=False)
val_dataset = get_data(val, shape = (256, 256), batch_size=64, shuffle=False, repeat=False)
test_dataset = get_data(test, shape = (256, 256), batch_size=64, shuffle=False, repeat=False)

In [16]:
def get_model(model_name):
    with strategy.scope():
        model = sm.Unet(model_name)
        model.compile(
            'SGD',
            loss = sm.losses.bce_jaccard_loss,
            metrics = [sm.metrics.iou_score]
        )
    return model

In [122]:
# with strategy.scope():
#     model = get_model('inceptionv3')
#     model.load_weights('../../ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_64_is_256/19.h5')

In [24]:
model.evaluate(test_dataset, verbose=1)

31/31 [==============================] - 7s 65ms/step - loss: 0.2183 - iou_score: 0.7612


[0.21827682852745056, 0.7612173557281494]

In [25]:
model.evaluate(train_dataset, verbose=1)

2949/2949 [==============================] - 156s 53ms/step - loss: 0.2184 - iou_score: 0.7841


[0.21842361986637115, 0.7840819954872131]

In [26]:
model.evaluate(val_dataset, verbose=1)

30/30 [==============================] - 2s 64ms/step - loss: 0.2215 - iou_score: 0.7813


[0.22150203585624695, 0.7812630534172058]

In [28]:
# dd5f438e8.jpg

In [17]:
def check_nan(data):
    if type(data[0]) == str:
        return True
    else:
        return False

In [18]:
train['updated_col'] = train['EncodedPixels'].apply(check_nan)

In [19]:
updated_train = train[train['updated_col'] == True]

In [21]:
updated_train = updated_train[:100]

In [23]:
updated_train_dataset = get_data(updated_train, shape = (256, 256), batch_size=64, shuffle=False, repeat=False)

In [24]:
with strategy.scope():
    model = get_model('inceptionv3')
    model.load_weights('../../ml-data-training/ship_seg_weights/segmentation/incep_baseline_bs_64_is_256/19.h5')

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [25]:
updated_train_pred = model.predict(updated_train_dataset, verbose=1)

2/2 [==============================] - 4s 280ms/step


In [93]:
pred_mask = PIL.Image.fromarray((np.where(updated_train_pred[10].reshape(256, 256) > 0.5, 1, 0) * 255).astype(np.uint8))
pred_mask = np.array(pred_mask).flatten()
pred_mask = np.where(updated_train_pred[10].reshape(256, 256) > 0.5, 1, 0).flatten()

In [110]:
g_truth = PIL.Image.open(updated_train['mask_paths'].values.tolist()[10])
g_truth = g_truth.resize((256, 256))
# g_truth = np.array(g_truth).flatten() / 255
g_truth = np.array(g_truth).flatten()
test_check = np.where(np.logical_and(g_truth >= 0 , g_truth < 255), 0, g_truth)
test_check = test_check / 255

In [120]:
TP = sum(np.logical_and(test_check == 1, pred_mask == 1))
FP = sum(np.logical_and(test_check == 0, pred_mask == 1))
FN = sum(np.logical_and(test_check == 1, pred_mask == 0))

In [121]:
TP / (TP + FP + FN)

0.4864864864864865